In [17]:
#install depencies!!!
import os
!pip install open3d
!pip install laspy
!pip install pillow

# Get the current working directory of the notebook and set the working directory to the notebook directory
notebook_dir = os.getcwd()
os.chdir(notebook_dir)
%cd $notebook_dir
print("Current folder:"+os.getcwd())

# Clone the yolov5 repository to current folder and install
!git clone https://github.com/ultralytics/yolov5
# Navigate to the yolov5 directory and install requirements
%cd yolov5
!pip install -r requirements.txt
%cd $notebook_dir


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mikapehk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mikapehk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


c:\LocalData\mikapehk\whorlDetector
Current folder:c:\LocalData\mikapehk\whorlDetector



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mikapehk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\mikapehk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\LocalData\mikapehk\whorlDetector\yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.


c:\LocalData\mikapehk\whorlDetector



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mikapehk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [49]:
# Renders TLS images of given tree point cloud sawlog part in four cardinal directions into TLSimages folder. 
# Give .las pointcloud path, bottom and top heights of the sawlog in the pointcloud and location of the stem in the point cloud (x,y)
# Makes the whorl detection using yolov5 model and saves the detections into detections/run folder.
# Combines detections of the different cardinal directions and linearly clusters stem using linear clustering streshold.
# Calculates the final whorl cluster count into whorlCount variable and mean whorl distance into meanWhorlDistance variable.

##import depencies
import laspy
import pandas as pd
import numpy as np
from statistics import mean
import os
import shutil
from PIL import Image
notebook_dir = os.getcwd() #set working directory (default ipynb location)
os.chdir(notebook_dir)
%cd $notebook_dir
print("Current folder:"+os.getcwd())

linearClusteringTreshold = 0.09 #set linear clustering streshold (cm): how stricly whorl detections of the different direction and close to together are merged
TreePointCloudFolder = notebook_dir+"/point_clouds/" #tree point cloud locations
treeLocations = pd.read_excel(notebook_dir+'/tree_log_locations.xlsx', skiprows=0) #locations of the trees and logs
model = notebook_dir+"\whorlDetector.pt" #give location of the trained model
TLSimages = notebook_dir+"\TLSimages" #location of to be rendered TLS images
model = notebook_dir+"\whorlDetector.pt" #location of the trained yolov5 model
whorl_df = pd.DataFrame(columns=['tree', 'log', 'whorl_count' , 'mean_whorl_distance']) #final whorl dataframe


for index, row in treeLocations.iterrows(): #loop through logs in treelocations df
    import open3d as o3d
    #get log specs
    tree_id = str(int(row['tree']))
    log_id = str(int(row['log']))
    logBottomHeight = row['log_bottom_h'] 
    logTopHeight = row['log_top_h'] 
    logLenght = logTopHeight-logBottomHeight
    stemLocationX = row['tree_X'] 
    stemLocationY = row['tree_Y'] 
    TreePointCloud = TreePointCloudFolder+tree_id+'.las'
    if not os.path.exists(TreePointCloud):  # Check if file exists
        print(f"File {TreePointCloud} not found. Stopping loop.")
        break  # Stop the loop if the file is missing
    # read point cloud
    las = laspy.read(TreePointCloud)
    new_file = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
    new_file.points = las.points
    las = new_file
    #cut the sawlog from point cloud with bottom and top heights
    new_file.points = new_file.points[las.z >= (logBottomHeight)] 
    new_file.points = new_file.points[las.z <= (logTopHeight)]
    #get XY middle point of the point cloud
    middleX = stemLocationX
    middleY = stemLocationY
    #Cut points 0.5 meter away from middle
    new_file.points = las.points[las.x <= middleX+0.50]
    new_file.points = las.points[las.x >= middleX-0.50]
    new_file.points = las.points[las.y <= middleY+0.50]
    new_file.points = las.points[las.y >= middleY-0.50]
    las = new_file
    print(f"Number of points: {len(las.points)}")  # Print the number of points
    #create open3d cloud
    point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))
    geom = o3d.geometry.PointCloud()
    geom.points = o3d.utility.Vector3dVector(point_data)
    #color the points to white
    geom.paint_uniform_color([1, 1, 1])
    #create visualizer
    visualizer = o3d.visualization.Visualizer()
    visualizer.create_window(window_name='Open3D', width=1000*2, height=1000*3, left=0, top=0, visible=True)
    #add cloud to the visualizer
    visualizer.add_geometry(geom)
    #load render style file e.g., background is black...
    render = visualizer.get_render_option()
    render.background_color = np.array([0, 0, 0])  # Black background
    render.point_size = 5  # Adjust point size if necessary
    #render.load_from_json("render.json")
    #control the view
    view_ctl = visualizer.get_view_control()
    #set orthographic projection
    view_ctl.change_field_of_view(-60)
    #set zoom of view
    view_ctl.set_zoom(0.55)
    #visualizer.run() #uncomment if you want interact with clouds
    # 1. dir image capture
    view_ctl.set_up((1, 0, 0))  
    view_ctl.set_up((0, 0, 1)) 
    view_ctl.set_front((0, 1, 0)) 
    visualizer.capture_screen_image(TLSimages+"/"+tree_id+'_'+log_id+"_1.png", do_render=True)
    # 2. dir image capture
    view_ctl.set_up((1, 0, 0))  
    view_ctl.set_up((0, 0, 1))  
    view_ctl.set_front((-1, 0, 0))  
    visualizer.capture_screen_image(TLSimages+"/"+tree_id+'_'+log_id+"_2.png", do_render=True)
    # 3. dir image capture
    view_ctl.set_up((1, 0, 0)) 
    view_ctl.set_up((0, 0, 1))  
    view_ctl.set_front((0, -1, 0))  
    visualizer.capture_screen_image(TLSimages+"/"+tree_id+'_'+log_id+"_3.png", do_render=True)
    # 4. dir image capture
    view_ctl.set_up((1, 0, 0))  
    view_ctl.set_up((0, 0, 1))  
    view_ctl.set_front((1, 0, 0))  
    visualizer.capture_screen_image(TLSimages+"/"+tree_id+'_'+log_id+"_4.png", do_render=True)

    # Crop black bars of the images
    folder_path = TLSimages # Path to your folder
    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):  # Process only PNG files
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            image_array = np.array(image)
            # Find non-black pixels (assuming black pixels have RGB values [0, 0, 0])
            non_black_pixels = np.any(image_array != [0, 0, 0], axis=-1)
            # Get the bounding box of non-black pixels
            rows = np.any(non_black_pixels, axis=1)
            cols = np.any(non_black_pixels, axis=0)
            # Crop the image by the bounding box
            top, bottom = np.argmax(rows), len(rows) - np.argmax(rows[::-1]) - 1
            left, right = np.argmax(cols), len(cols) - np.argmax(cols[::-1]) - 1
            cropped_image = image.crop((left, top, right, bottom))
            # Save the cropped image with the same filename
            cropped_image.save(os.path.join(folder_path, f"{filename}"))

    #destroy open3d objects or error
    del o3d, visualizer, view_ctl, render

    #delete previous detections if exist
    detections = notebook_dir+"\detections"
    if os.path.exists(detections) and os.path.isdir(detections):
        shutil.rmtree(detections)

    #make the whorl detections on the TLS images
    %cd yolov5
    !python detect.py --source {TLSimages} --weights {model} --project {detections} --name run  --save-txt --conf 0.30
    %cd $notebook_dir

    # detection label folder files
    labelDirection = detections+"/run\labels"
    label_files = os.listdir(labelDirection)
    whorls=[]

    # Loop through each direction in the directory
    for filename in os.listdir(labelDirection):
        label = os.path.join(labelDirection, filename)
        # Read the detections CSV file
        detectionLabels = pd.read_csv(label, sep=' ', engine='python', header=None)
        detectionsDeNormalized = ((1-detectionLabels[2]) * (logLenght))+logBottomHeight
        #add directions detections to the whorl list
        whorls += detectionsDeNormalized.tolist()

    # Cluster whorl detections linearly
    whorls.sort()
    n = linearClusteringTreshold
    numbers = whorls
    nnumbers = np.array(numbers)
    clusters = pd.DataFrame({
    'numbers': numbers,
    'segment': np.cumsum([0] + list(1*(nnumbers[1:] - nnumbers[0:-1] > n))) + 1
    }).groupby('segment').agg({'numbers': set}).to_dict()['numbers']
    klust = list()
    for x in range(len(clusters)):
        klust.append(mean(clusters[x+1]))
    whorls = klust
    whorls.sort()

    #calculate whorl count
    whorlCount = len(whorls)
    # calculate mean distance between whorls
    meanDist=list()
    if(len(whorls)>1):
        for x in range(len(whorls)-1):
            meanDist.append(whorls[x+1]-whorls[x])
        meanWhorlDistance = mean(meanDist)*100
    else:
        #if less than 1 whorl set -1
        meanWhorlDistance = -1

    print("Number of whorls: " + str(whorlCount))
    print("Mean distance between whorls: " + str(meanWhorlDistance) + " cm")

    whorl_df.loc[len(whorl_df)] = [tree_id, log_id, whorlCount, meanWhorlDistance] #add data to df

print("Job is done!")
whorl_df.to_csv(notebook_dir+"/whorls.csv", index=False)  # Saves without the index column


C:\Users\mikapehk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\LocalData\mikapehk\whorlDetector
Current folder:c:\LocalData\mikapehk\whorlDetector
Number of points: 32432
c:\LocalData\mikapehk\whorlDetector\yolov5


C:\Users\mikapehk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
detect: weights=['c:\\LocalData\\mikapehk\\whorlDetector\\whorlDetector.pt'], source=c:\LocalData\mikapehk\whorlDetector\TLSimages, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=c:\LocalData\mikapehk\whorlDetector\detections, name=run, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-393-g6981c274 Python-3.9.13 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 322 layers

c:\LocalData\mikapehk\whorlDetector
Number of whorls: 20
Mean distance between whorls: 25.221076911105268 cm
Number of points: 7363
c:\LocalData\mikapehk\whorlDetector\yolov5


C:\Users\mikapehk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
detect: weights=['c:\\LocalData\\mikapehk\\whorlDetector\\whorlDetector.pt'], source=c:\LocalData\mikapehk\whorlDetector\TLSimages, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=c:\LocalData\mikapehk\whorlDetector\detections, name=run, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-393-g6981c274 Python-3.9.13 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 322 layers

c:\LocalData\mikapehk\whorlDetector
Number of whorls: 18
Mean distance between whorls: 24.854976089757955 cm
Number of points: 18484
c:\LocalData\mikapehk\whorlDetector\yolov5


C:\Users\mikapehk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\LocalData\mikapehk\whorlDetector
Number of whorls: 18
Mean distance between whorls: 24.778070561558817 cm
Job is done!


detect: weights=['c:\\LocalData\\mikapehk\\whorlDetector\\whorlDetector.pt'], source=c:\LocalData\mikapehk\whorlDetector\TLSimages, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=c:\LocalData\mikapehk\whorlDetector\detections, name=run, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-393-g6981c274 Python-3.9.13 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
image 1/12 C:\LocalData\mikapehk\whorlDetector\TLSimages\1022_2_1.png: 640x160 16 knotKlusters, 145.5ms
image 2/12 C:\LocalData\mikapehk\whorlDetector\TLSimages\1022_2_2.png: 640x160 20 knotKlusters, 136.0ms
image 3/12 C:\LocalData\mikapehk\whorlDetector\

In [40]:
import laspy
import numpy as np
import matplotlib.pyplot as plt

def render_point_cloud_image(TreePointCloud, output_image_path):
    """
    Function to render an image of a point cloud with black background and white points.
    The image borders will be based on the points' boundaries.
    
    Parameters:
        TreePointCloud (str): Path to the .las file containing the point cloud data.
        output_image_path (str): Path to save the rendered image.
    """
    # Read the point cloud using laspy
    las = laspy.read(TreePointCloud)
    
    # Extract point data (x, y, z coordinates)
    points = np.vstack([las.x, las.y, las.z]).T
    
    # Get the min and max values of x and y for setting the boundaries
    x_min, x_max = points[:, 0].min(), points[:, 0].max()
    y_min, y_max = points[:, 1].min(), points[:, 1].max()

    # Create a 2D projection of the points (ignoring Z axis)
    plt.figure(figsize=(10, 10))
    
    # Scatter plot the points with white color
    plt.scatter(points[:, 0], points[:, 1], c='white', s=1)

    # Set the background color to black
    plt.gca().set_facecolor('black')

    # Set axis limits based on point cloud's min/max values
    plt.xlim(x_min - 1, x_max + 1)
    plt.ylim(y_min - 1, y_max + 1)

    # Hide axes for better visualization
    plt.axis('off')
    
    # Save the image
    plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0, dpi=300)
    
    # Close the plot
    plt.close()
    
    print(f"Image saved at: {output_image_path}")

# Example usage:
TreePointCloud = "C:\LocalData\mikapehk\whorlDetector\point_clouds/104.las"  # Replace with actual file path
output_image_path = "C:\LocalData\mikapehk\whorlDetector\point_clouds/output_image.png"  # Replace with desired output path
render_point_cloud_image(TreePointCloud, output_image_path)


Image saved at: C:\LocalData\mikapehk\whorlDetector\point_clouds/output_image.png
